# k8s环境搭建

## 目标

搭建一个多master节点、多worker节点集群环境，用于学习kubernets的架构（公司的kubernetes环境受限，无法使用k8s新特性），并将自己的应用服务迁移部署到k8s集群内。

## 环境准备

etcd集群至少需要3个节点，因此master高可用集群也至少需要3个节点，另外配置2个工作节点。

华为云等云服务提供商租赁虚拟机VM的成本比较高，因此在物理上通过VirtualBox虚拟化技术虚拟VM。

### VirtualBox构建虚拟VM

物理机软、硬件环境：

|资源|配置|
|---|---|
|CPU|20 Core|
|内存|64GB|
|磁盘|SSD 1T、HHD 2T|
|操作系统|Ubuntu 18.04|

网络要求：

* 虚拟机之间能互相通信
* 虚拟机能访问公网

**开始使用NAT Network网络模式，但过段时间后物理机和虚拟机之间无法进行网络通信，后面将网络模式切换为桥接模式。**

软件版本：

|软件|版本|
|---|---|
|VirtualBox|`6.0.24`|
|镜像|`Ubuntu 18.04`|

> **VirtualBox尽量匹配上述版本。**

实操过程省略。

### 配置虚拟机免密登录

机器列表：

`/etc/hosts`

```
# vms
172.30.12.169 vm1
172.30.12.170 vm2
172.30.12.171 vm3
172.30.12.172 vm4
172.30.12.175 vm5
```

VM1作为主要的ssh登录机器，能通过ssh免密登录到其他所有机器。

1. VM安装`openssh-server`
2. 配置VM1免密登录其他所有VM

实操过程省略。

### 配置虚拟机sudo免密输入

更新`/etc/sudoers`文件

sudo vim `/etc/sudoers`

```
user_name ALL=(ALL:ALL) ALL
user_name ALL=(ALL) NOPASSWD: ALL
```

### 配置物理机登录虚拟机

虚拟机到物理机的ssh隧道：

1. 虚拟机安装autossh `sudo apt install autossh`，创建ssh隧道 `autossh -NfR 9999:localhost:22 user_name@ip`
2. 物理机安装sshpass `sudo apt install sshpass`，登录到虚拟机 `sshpass -p user_name ssh localhost -p9999`

> 网络模式从**NAT Network**改成**桥接**模式后，4网络是互通，不再需要端口或ssh隧道。

### 更新Ubuntu软件源

参考清华大学开源软件镜像站: https://mirrors.tuna.tsinghua.edu.cn/help/ubuntu/

初始化系统环境

```shell
sudo apt-get -y install build-essential libssl-dev git zsh

```

### 安装ohmyzsh

参考https://github.com/ohmyzsh/ohmyzsh

## 搭建k8s集群

### 安装docker

参考

* https://phoenixnap.com/kb/how-to-install-docker-on-ubuntu-18-04
* *docker notebook*